In [1]:
import keras
import tensorflow as tf
import numpy as np

Using TensorFlow backend.


In [2]:
# Note: data isn't in the repository

# loading data
data = np.load("data/x_images.npy")
labels = np.load("data/y_images.npy")

# shuffling data
indices = np.random.choice(np.arange(len(data)), size=len(data), replace=False)
data = data[indices]
labels = labels[indices]

# encode labels to one-hot
num_classes = len(set((labels)))
labels = keras.utils.to_categorical(labels, num_classes)

print(data.shape)
print(labels.shape)

(234, 288, 352, 3)
(234, 8)


In [3]:
# load a pretrained model (without the top dense layers)

base_model = keras.applications.xception.Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=[288,352,3], pooling=None, classes=num_classes)

In [4]:
# Freeze the layers except the last 2 layers, so the weights do not change
# (number of layers can vary)
for layer in base_model.layers[:-2]:
    layer.trainable = False


In [5]:
# add layers to the pretrained network
model = keras.Sequential()
model.add(base_model)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.7))
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dropout(0.6))
model.add(keras.layers.Dense(num_classes, activation='softmax'))

In [6]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=1e-8),
              metrics=['acc'])
# Train the model
history = model.fit(x=data,y=labels,batch_size=50,epochs=100,validation_split=0.3)


Train on 163 samples, validate on 71 samples
Epoch 1/100
163/163 [==============================] - 10s 59ms/step - loss: 3.0989 - acc: 0.1104 - val_loss: 11.2054 - val_acc: 0.1549
Epoch 2/100
163/163 [==============================] - 2s 10ms/step - loss: 3.0845 - acc: 0.0982 - val_loss: 11.2254 - val_acc: 0.1408
Epoch 3/100
163/163 [==============================] - 2s 10ms/step - loss: 3.0763 - acc: 0.0859 - val_loss: 11.2262 - val_acc: 0.1408
Epoch 4/100
163/163 [==============================] - 2s 10ms/step - loss: 3.2891 - acc: 0.0859 - val_loss: 11.2269 - val_acc: 0.1408
Epoch 5/100
163/163 [==============================] - 2s 10ms/step - loss: 3.0315 - acc: 0.1288 - val_loss: 11.2201 - val_acc: 0.1408
Epoch 6/100
163/163 [==============================] - 2s 11ms/step - loss: 2.9659 - acc: 0.0798 - val_loss: 11.2209 - val_acc: 0.1549
Epoch 7/100
163/163 [==============================] - 2s 11ms/step - loss: 3.0761 - acc: 0.0859 - val_loss: 11.2205 - val_acc: 0.1549
Epoch 8/1

Epoch 61/100
163/163 [==============================] - 2s 10ms/step - loss: 3.0392 - acc: 0.1288 - val_loss: 11.1389 - val_acc: 0.1549
Epoch 62/100
163/163 [==============================] - 2s 10ms/step - loss: 2.7727 - acc: 0.1718 - val_loss: 11.1381 - val_acc: 0.1549
Epoch 63/100
163/163 [==============================] - 2s 11ms/step - loss: 2.8547 - acc: 0.1043 - val_loss: 11.1359 - val_acc: 0.1549
Epoch 64/100
163/163 [==============================] - 2s 11ms/step - loss: 2.9231 - acc: 0.1350 - val_loss: 11.1350 - val_acc: 0.1549
Epoch 65/100
163/163 [==============================] - 2s 11ms/step - loss: 3.1155 - acc: 0.1043 - val_loss: 11.1351 - val_acc: 0.1549
Epoch 66/100
163/163 [==============================] - 2s 11ms/step - loss: 3.0584 - acc: 0.1104 - val_loss: 11.1348 - val_acc: 0.1549
Epoch 67/100
163/163 [==============================] - 2s 10ms/step - loss: 2.8267 - acc: 0.1104 - val_loss: 11.1354 - val_acc: 0.1549
Epoch 68/100
163/163 [==========================

In [16]:
# save the model
val_acc = str(int(history.history["val_acc"][-1]*100))
val_loss = str(int(history.history["val_loss"][-1]*10))
filename = 'data/model/model_val_acc_'+val_acc+"_val_loss_"+val_loss+'.h5'
model.save(filename)